# Model

This Notebook Shows how to build a Model using TensorFlow and how to train it

## Library

In [ ]:
import librosa
import numpy as np
import tensorflow as tf
import matplotplib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score

## Model Architecture

There are 4 Models that is being used in my thesis that can be made using TensorFlow Libraries.

### VGGNet16

In [ ]:
# Initialize TensorFlow Sequential
model = tf.keras.Sequential()

# You could create VGGNet from Scratch (Read the Original Paper) or just use the built in architecture from TensorFlow which iis the same
# weights should be set if want to use transfer learning
# include_top should be false as we use different classifier
# input shape should be (MFCCs Coefficient, Length of the Frame, 1)
base_model = tf.keras.applications.vgg16.VGG16(
    include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation="relu"))
model.add(tf.keras.layers.Dense(512, activation="relu"))
# The last layer uses 6 as parameter for 6 classes 
model.add(tf.keras.layers.Dense(6, activation="softmax"))

### ResNet50v2

In [ ]:
# Initialize TensorFlow Sequential
model = tf.keras.Sequential()

# You could create ResNet50v2 from Scratch (Read the Original Paper) or just use the built in architecture from TensorFlow which iis the same
# weights should be set if want to use transfer learning
# include_top should be false as we use different classifier
# input shape should be (MFCCs Coefficient, Length of the Frame, 1)
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))

model.add(tf.keras.layers.Flatten())
# The last layer uses 6 as parameter for 6 classes 
model.add(tf.keras.layers.Dense(6, activation="softmax"))

### ResNet50v2-LSTM

In [ ]:
# Initialize TensorFlow Sequential
model = tf.keras.Sequential()

# You could create ResNet50v2 from Scratch (Read the Original Paper) or just use the built in architecture from TensorFlow which iis the same
# weights should be set if want to use transfer learning
# include_top should be false as we use different classifier
# input shape should be (MFCCs Coefficient, Length of the Frame, 1)
base_model = tf.keras.applications.resnet_v2.ResNet50V2(
    include_top=False, weights=None, input_shape=(train_data_value.shape[1], train_data_value.shape[2], 1))

# Flatten the Matrixes with Time Distributed Properties
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))

# The Bidirectional LSTM Layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)))

# The last layer uses 6 as parameter for 6 classes 
model.add(tf.keras.layers.Dense(6, activation="softmax"))

### CNN-LSTM (Main Model)

In [ ]:
# Initialize TensorFlow Sequential
model = tf.keras.Sequential()

# input shape should be (MFCCs Coefficient, Length of the Frame, 1)
# The following model uses 4-block CNN which extract features from MFCCs and then fed into Bi-LSTM layer before the final classifier

# 1st CNN-Block
model.add(tf.keras.layers.Conv2D(64, (3, 3), input_shape=(
    train_data_value.shape[1], train_data_value.shape[2], 1)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.MaxPooling2D(
    (2, 2), strides=(2, 2), padding='same'))

# 2nd CNN-Block
model.add(tf.keras.layers.Conv2D(64, (3, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.MaxPooling2D(
    (2, 2), strides=(2, 2), padding='same'))

# 3rd CNN-Block
model.add(tf.keras.layers.Conv2D(128, (2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))
model.add(tf.keras.layers.MaxPooling2D(
    (2, 2), strides=(2, 2), padding='same'))

# 4th CNN-Block
model.add(tf.keras.layers.Conv2D(128, (2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('elu'))

# Flatten the Matrixes with Time Distributed Properties
model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Flatten()))

# The Bidirectional LSTM Layer
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)))

# The last layer uses 6 as parameter for 6 classes 
model.add(tf.keras.layers.Dense(6, activation="softmax"))

## Model Training

The following section show on how to train the model

### Setting up Optimizer, Learning Rate, and Helper

In [ ]:
# Assigning Optimizer to Model
optimiser = tf.keras.optimizers.get("adam")

# Assigning Learning Rate to Model
optimiser.learning_rate.assign(0.0001)

# For more complete information about optimizer you could view the TensorFlow Documentation

# Early Stop Function that Track Validation Loss
# This will stop the training if the validation loss does not improve
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=5)

# This will be used as training calllback to save the log
csv_logger = tf.keras.callbacks.CSVLogger('Model_Log')

### Training

In [ ]:
# For more Complete guide see the TensorFlow Documentation
# You Could use validation_split=0.8 instead of validation data if you dont split it manually
# batch_size refer to data looped/step. set it as 32 for standard value, increase it to increase performance
# epochs refers to the number of training loop
# callbacks should filled with early stopping and csv logger callback (or your additional callbacks)
history = model.fit(train_data_value, train_data_target, validation_data=(
    validation_data_value, validation_data_target), batch_size=32, epochs=60, callbacks=[csv_logger, early_stop])

# Saving the model
model.save("model")

## Model Performance

Using the previous trained model from the Model Training Section, we can see the performance model with various metrics.

### Accuracy Test

In [ ]:
# Load Test Dataset
test_data_value = np.load("test_data_value.npy")
test_data_target = np.load("test_data_target.npy")

# if uses saved model
# model = tf.keras.models.load_model("...") 

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_data_value, test_data_target, verbose=0)

# Get the prediction for the test data
test_pred = model.predict(test_data_value)

# Compute the overall accuracy/prediction
test_pred_classes = np.argmax(test_pred, axis=1)
test_true_classes = np.argmax(test_data_target, axis=1)
test_accuracy = np.mean(test_pred_classes == test_true_classes)
print(test_accuracy)

### Confusion Matrix, F1-Score, Recall, And Precision

In [ ]:
# Load Saved Model
loaded_model = tf.keras.models.load_model("...")

# Load Test Dataset
test_data_value = np.load("test_data_value.npy")
test_data_target = np.load("test_data_target.npy")

predictions = loaded_model.predict(test_data_value)
predicted_labels = np.argmax(predictions, axis=1)
confusion_matrix = tf.math.confusion_matrix(test_data_target, predicted_labels)

test_loss, test_acc = loaded_model.evaluate(test_data_value, test_data_target)

y_pred = loaded_model.predict(test_data_value)
y_pred = tf.argmax(y_pred, axis=1)
y_true_tensor = tf.convert_to_tensor(test_data_target)

# Confusion Matrix
confusion_mtx = tf.math.confusion_matrix(y_true_tensor, y_pred)

# Normalized Version
confusion_array = confusion_mtx.numpy()
row_sums = confusion_array.sum(axis=1, keepdims=True)
normalized_confusion_array = confusion_array / row_sums


# The Following Line is to Plot the Confusion Matrix into Heatmap
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

axs[0].set_title('Confusion Matrix')
sns.heatmap(confusion_mtx,
            xticklabels=["disgust", "happy", "sad", "neutral", "fear", "angry"],
            yticklabels=["disgust", "happy", "sad", "neutral", "fear", "angry"],
            annot=True, fmt='g', cbar=False, ax=axs[0], annot_kws={"fontsize": 12})
axs[0].set_xlabel('Prediction', fontsize=12)
axs[0].set_ylabel('Label', fontsize=12)
axs[0].set_xticklabels(axs[1].get_xticklabels(), fontsize=12)
axs[0].set_yticklabels(axs[1].get_yticklabels(), fontsize=12)

axs[1].set_title('Normalized Confusion Matrix')
sns.heatmap(normalized_confusion_array,
            xticklabels=["disgust", "happy", "sad", "neutral", "fear", "angry"],
            yticklabels=["disgust", "happy", "sad", "neutral", "fear", "angry"],
            annot=True, fmt='.2%', cbar=False, ax=axs[1], annot_kws={"fontsize": 12})
axs[1].set_xlabel('Prediction', fontsize=12)
axs[1].set_ylabel('Label', fontsize=12)
axs[1].set_xticklabels(axs[1].get_xticklabels(), fontsize=12)
axs[1].set_yticklabels(axs[1].get_yticklabels(), fontsize=12)
plt.tight_layout()
plt.show()

F1-Score, Recall, And Precision

In [ ]:
# Load Saved Model
loaded_model = tf.keras.models.load_model("...")

# Load Test Dataset
test_data_value = np.load("test_data_value.npy")
test_data_target = np.load("test_data_target.npy")

# Predict
predictions = loaded_model.predict(test_data_value)  
predicted_labels = tf.argmax(predictions, axis=1)

# Get the Precision, Recall, and F1
precision = precision_score(test_data_target, predicted_labels, average=None)
recall = recall_score(test_data_target, predicted_labels, average=None)
f1 = f1_score(test_data_target, predicted_labels, average=None)

# Print the Precision, Recall, and F1
print("Each Class")
print("Precision:{}".format(precision))
print("Recall:{}".format(recall))
print("F1:{}".format(f1))